<a href="https://colab.research.google.com/github/kritika-gupta/IE406-machine-learning/blob/main/assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import multivariate_normal

#### 1. Perform Linear Discriminant Analysis (LDA) on the MNIST dataset for binary as well as formulticlass classification.

#### 2. Perform Quadratic Discriminant Analysis(QDA) on the MNIST dataset for multiclass classification. 

In [ ]:
digits = load_digits()
digits.keys()
digits.data.shape

(1797, 64)

In [ ]:
labels = digits.target
data = digits.data[:]
print(data.shape, labels.shape)
data = StandardScaler().fit_transform(data)

(1797, 64) (1797,)


## Q1 . Binary Classification (class 0 and class 1)

In [ ]:
idxs = np.where((labels==0) | (labels==1))[0]
X = data[idxs, :]
Y = labels[idxs]
print(X.shape, Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.9)

(360, 64) (360,)


In [ ]:
x0 = X_train[np.where(Y_train==0)]
y0 = Y_train[np.where(Y_train==0)]
x1 = X_train[np.where(Y_train==1)]
y1 = Y_train[np.where(Y_train==1)]
print(x0.shape, y0.shape, x1.shape, y1.shape)

(158, 64) (158,) (166, 64) (166,)


In [ ]:
cov0 = np.cov(x0.T)
cov1 = np.cov(x1.T)
mu0 = np.mean(x0, axis = 0).reshape(x0.shape[1], 1)
mu1 = np.mean(x1, axis = 0).reshape(x0.shape[1], 1)
print(cov0.shape, mu0.shape, cov1.shape, mu1.shape)
cov = (cov0+cov1)/2
print(cov.shape)
# priori
prior0 = len(y0)/(len(y0)+len(y1))
prior1 = len(y1)/(len(y0)+len(y1))

(64, 64) (64, 1) (64, 64) (64, 1)
(64, 64)


In [ ]:
cov_inv = np.linalg.pinv(cov)
beta = cov_inv@mu1 - cov_inv@mu0
b = 0.5*(mu0.T@cov_inv@mu0 - mu1.T@cov_inv@mu1) - np.log(prior0/prior1)
print(beta.shape, b.shape)

(64, 1) (1, 1)


In [ ]:
pred = np.where(X_test@beta + b > 0, 1, 0)
pred = pred.reshape(pred.shape[0],)
accuracy = np.sum(Y_test==pred)/len(pred)
print("Accuracy = ", accuracy*100, "%")

Accuracy =  100.0 %


## Q2. Multiclass Classification

In [ ]:
def delta_lda(i, x):
    return multivariate_normal.pdf(x, mu[i], sigma, allow_singular=True)*priori[i]

def predict_lda(x):
    return np.argmax([delta_lda(i, x) for i in digits.target_names])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, train_size = 0.9)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
X_vals = [X_train[np.where(Y_train==i)] for i in digits.target_names] # list of Xs for each class
Y_vals = [Y_train[np.where(Y_train==i)] for i in digits.target_names]

(1617, 64) (1617,) (180, 64) (180,)


In [ ]:
sigma = np.array([np.cov(X_vals[i].T) for i in digits.target_names])
sigma = np.mean(sigma, axis = 0)
print(sigma.shape)
mu = [np.mean(X_vals[i], axis = 0) for i in digits.target_names]
total_len = len([x for sublist in Y_vals for x in sublist])
priori = [len(Y_vals[i])/total_len for i in digits.target_names]

(64, 64)


In [ ]:
accuracy = np.sum([predict_lda(X_test[j, :])==Y_test[j] for j in range(len(Y_test))])
print("Accuracy =", accuracy*100/len(Y_test), "%")

Accuracy = 93.33333333333333 %


# Q2. QDA

In [ ]:
def delta_qda(i, x):
    return multivariate_normal.pdf(x, mu[i], sigma[i], allow_singular=True)*priori[i]
def predict_qda(x):
    return np.argmax([delta_qda(i, x) for i in digits.target_names])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, train_size = 0.9)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
X_vals = [X_train[np.where(Y_train==i)] for i in digits.target_names] # list of Xs for each class
Y_vals = [Y_train[np.where(Y_train==i)] for i in digits.target_names]

(1617, 64) (1617,) (180, 64) (180,)


In [ ]:
sigma = [np.cov(X_vals[i].T) for i in digits.target_names]
mu = [np.mean(X_vals[i], axis = 0) for i in digits.target_names]
total_len = len([x for sublist in Y_vals for x in sublist])
priori = [len(Y_vals[i])/total_len for i in digits.target_names]

In [ ]:
accuracy = np.sum([predict_qda(X_test[j, :])==Y_test[j] for j in range(len(Y_test))])
print("Accuracy for QDA =", accuracy*100/len(Y_test), "%")

Accuracy for QDA = 95.55555555555556 %
